# Inicializando o animador

In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from dataclasses import dataclass
import random
import math
import os

@dataclass
class Point:
    x: float
    y: float

class ConvexHullAnimator:
    def __init__(self, points):
        self.points = points
        self.hull = []
        self.frames = []

    def save_frame(self, current_point):
        frame = {
            "hull": list(self.hull),
            "points": list(self.points),
            "current": current_point
        }
        self.frames.append(frame)

    def animate(self, test_id):
        fig, ax = plt.subplots(figsize=(6, 6))

        def update(i):
            if i >= len(self.frames) - 3:
                i = len(self.frames) - 1

            ax.clear()
            frame = self.frames[i]
            pts = frame["points"]
            hull = frame["hull"]
            current = frame["current"]

            ax.set_title(f"Etapa {i}")
            ax.set_xlim(min(p.x for p in pts) - 1, max(p.x for p in pts) + 1)
            ax.set_ylim(min(p.y for p in pts) - 1, max(p.y for p in pts) + 1)

            ax.scatter([p.x for p in pts], [p.y for p in pts], color='blue', label='Pontos')
            
            if current:
                ax.scatter(current.x, current.y, color='green', s=100, label='Novo ponto')

            if hull:
                hx = [p.x for p in hull] + [hull[0].x]
                hy = [p.y for p in hull] + [hull[0].y]
                ax.plot(hx, hy, color='red', label='Hull')

                for idx, p in enumerate(hull):
                    ax.text(p.x, p.y, str(idx), fontsize=12, ha='right', color='black', fontweight='bold')

            ax.legend()
            ax.grid(True)

        anim = FuncAnimation(fig, update, frames=len(self.frames) + 2, interval=800, repeat=False)
        os.makedirs("Simulações", exist_ok=True)
        anim.save(f"Simulações/convex_hull_{test_id}.mp4", writer="ffmpeg", fps=1)
        plt.close(fig)


# Gift-Wrapping

In [2]:
def orientation(p, q, r):
    val = (q.y - p.y) * (r.x - q.x) - (q.x - p.x) * (r.y - q.y)
    if val == 0: return 0
    return 1 if val > 0 else 2

def distance(p1, p2):
    return (p1.x - p2.x)**2 + (p1.y - p2.y)**2


def gift_wrapping(points, animator):
    if len(points) < 3:
        animator.hull = points
        animator.save_frame(None)
        return points

    # Começa pelo ponto mais abaixo
    start = min(points, key=lambda p: (p.y, p.x))
    current = start
    hull = []

    direction = Point(1, 0)  # vetor de referência inicial: eixo x positivo

    while True:
        hull.append(current)
        animator.hull = list(hull)
        animator.save_frame(current)

        next_point = None
        min_angle = math.inf

        for candidate in points:
            if candidate == current:
                continue

            dx = candidate.x - current.x
            dy = candidate.y - current.y
            angle = math.atan2(dy, dx)

            # Normalize o ângulo relativo ao vetor anterior
            # Fazemos isso para garantir que sempre giramos "para frente"
            rel_angle = (angle - math.atan2(direction.y, direction.x)) % (2 * math.pi)
            # Atualize próximo ponto apenas se o ângulo for menos anti-horário (mais horário)
            if rel_angle < min_angle:
                min_angle = rel_angle
                next_point = candidate
            elif rel_angle == min_angle:
                # Se ângulo igual, pega o mais distante
                if distance(current, candidate) > distance(current, next_point):
                    next_point = candidate

        if next_point == start or next_point is None:
            break

        # Atualiza direção e ponto atual
        direction = Point(next_point.x - current.x, next_point.y - current.y)
        current = next_point

    animator.hull = hull
    animator.save_frame(None)
    return hull

# Graham Scan

In [3]:
from collections import defaultdict
def polar_angle(p0, p1):
    return math.atan2(p1.y - p0.y, p1.x - p0.x)

def distance(p0, p1):
    return (p1.x - p0.x)**2 + (p1.y - p0.y)**2

def orientation_cross(p, q, r):
    """Retorna positivo se p→q→r faz curva à esquerda, 0 se colinear, negativo se direita."""
    return (q.x - p.x)*(r.y - p.y) - (q.y - p.y)*(r.x - p.x)

def graham_scan(points, animator):
    if len(points) < 3:
        animator.hull = points
        animator.save_frame(None)
        return points

    # 1. Ponto mais abaixo
    p0 = min(points, key=lambda p: (p.y, p.x))

    # 2. Agrupar pontos por ângulo polar e pegar apenas o mais distante em cada grupo
    angle_map = defaultdict(list)
    for p in points:
        if p == p0:
            continue
        angle = polar_angle(p0, p)
        angle_map[angle].append(p)

    filtered_pts = []
    for angle in sorted(angle_map.keys()):
        farthest = max(angle_map[angle], key=lambda p: distance(p0, p))
        filtered_pts.append(farthest)

    sorted_pts = [p0] + filtered_pts

    # 3. Inicializa pilha
    stack = [sorted_pts[0], sorted_pts[1]]
    animator.hull = list(stack)
    animator.save_frame(sorted_pts[1])

    # 4. Processar os demais pontos
    for p in sorted_pts[2:]:
        while len(stack) >= 2 and orientation_cross(stack[-2], stack[-1], p) <= 0:
            stack.pop()
            animator.hull = list(stack)
            animator.save_frame(p)

        stack.append(p)
        animator.hull = list(stack)
        animator.save_frame(p)

    # 5. Hull final
    animator.hull = stack
    animator.save_frame(None)
    return stack

# Rodando as simulações

In [4]:
if __name__ == "__main__":
    # Criando pontos aleatórios
    random.seed(4)
    points = [Point(random.randint(-10, 10), random.randint(-10, 10)) for _ in range(150)]
    print(points)
    # Gift Wrapping
    animator1 = ConvexHullAnimator(points)
    stack = gift_wrapping(points, animator1)
    print(stack)
    animator1.animate("gift_wrapping")

    # Graham Scan
    animator2 = ConvexHullAnimator(points)
    stack = graham_scan(points, animator2)
    print(stack)
    animator2.animate("graham_scan")


[Point(x=-3, y=-1), Point(x=-7, y=2), Point(x=5, y=-6), Point(x=-8, y=-8), Point(x=-10, y=2), Point(x=7, y=-1), Point(x=-9, y=-3), Point(x=6, y=7), Point(x=1, y=-2), Point(x=-5, y=-7), Point(x=-2, y=-4), Point(x=-10, y=10), Point(x=-2, y=-2), Point(x=-4, y=-5), Point(x=-1, y=-1), Point(x=10, y=1), Point(x=-8, y=9), Point(x=0, y=2), Point(x=6, y=-3), Point(x=-5, y=-3), Point(x=5, y=-2), Point(x=-8, y=7), Point(x=-1, y=-10), Point(x=-1, y=8), Point(x=-1, y=6), Point(x=-4, y=3), Point(x=3, y=9), Point(x=-1, y=3), Point(x=4, y=-5), Point(x=-3, y=-1), Point(x=-2, y=-9), Point(x=-8, y=-9), Point(x=4, y=10), Point(x=-2, y=6), Point(x=7, y=10), Point(x=5, y=0), Point(x=-6, y=-4), Point(x=-8, y=3), Point(x=-4, y=10), Point(x=10, y=4), Point(x=-2, y=-5), Point(x=1, y=3), Point(x=8, y=0), Point(x=10, y=7), Point(x=-4, y=0), Point(x=-7, y=-9), Point(x=-3, y=-2), Point(x=8, y=9), Point(x=-3, y=-7), Point(x=0, y=-5), Point(x=-1, y=4), Point(x=-10, y=-9), Point(x=1, y=-8), Point(x=-1, y=0), Point(x=-

# Para testar o algoritmo, mude a váriavel "pts" abaixo e rode o algoritmo. 
Se quiser uma simulação visual com o matplotlib, mude a variável "animated" para "True". Se quiser gerar pontos aleatórios, utilize a função "generate_random_points" como pode ser visto abaixo. Caso contrário, passe uma array de pontos no formato [(a,b), (c,d), (e,f), ... (y,z)]

In [5]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.uniform(-10, 10), random.uniform(-10, 10)) for _ in range(num_points)]

In [7]:
# ESSAS SÃO AS VARIÁVEIS QUE VOCÊ DEVE MUDAR
points = generate_random_points(105)
animated = True




# Criando classe fodder para o caso de não querer animação
class NullAnimator:
    def __init__(self, *args, **kwargs): pass
    def save_frame(self, _): pass
    def animate(self, _): pass
    hull = []
points = [Point(*pt) if not isinstance(pt, Point) else pt for pt in points]

animator1 = ConvexHullAnimator(points) if animated else NullAnimator()
animator2 = ConvexHullAnimator(points) if animated else NullAnimator()
stack = gift_wrapping(points, animator=animator1)
print(f"Convex Hull encontrado com Gift-Wrapping: {stack}")
if animated:
    animator1.animate(f"gift_wrapping_{len(points)}_points")
    print("Simulação salva em Simulações/convex_hull_gift_wrapping_{len(pts)}_points.mp4")


stack = graham_scan(points, animator=animator2)
print(f"Convex Hull encontrado com Graham Scan: {stack}")
if animated:
    animator2.animate(f"graham_scan_{len(points)}_points")
    print("Simulação salva em Simulações/convex_hull_graham_scan_{len(pts)}_points.mp4")

Convex Hull encontrado com Gift-Wrapping: [Point(x=3.079526354214652, y=-9.843537856956841), Point(x=6.994753892494639, y=-8.543435416308618), Point(x=8.784742806494314, y=-7.693164962971448), Point(x=9.408012220444558, y=-6.4286436765507275), Point(x=9.969088817088846, y=9.92192895710189), Point(x=9.146352817193463, y=9.908453789854278), Point(x=2.233555314519002, y=9.744661272630086), Point(x=-6.851345441210334, y=9.215578065489009), Point(x=-9.96617443562741, y=8.511503309981656), Point(x=-9.991881206054249, y=-4.261725662621457), Point(x=-9.52671130597097, y=-6.137404233445826), Point(x=-6.017401345468468, y=-9.511492245463472), Point(x=-1.4462616203864123, y=-9.806606007833201)]
Simulação salva em Simulações/convex_hull_gift_wrapping_{len(pts)}_points.mp4
Convex Hull encontrado com Graham Scan: [Point(x=3.079526354214652, y=-9.843537856956841), Point(x=6.994753892494639, y=-8.543435416308618), Point(x=8.784742806494314, y=-7.693164962971448), Point(x=9.408012220444558, y=-6.428643